Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4

## Assignment

- [ ] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [ ] [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website. You will be given the URL in Slack. Go to the Rules page. Accept the rules of the competition.
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your validation accuracy score.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

---

## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding. For example, you could try `quantity`, `basin`, `extraction_type_class`, and more. (But remember it may not work with high cardinality categoricals.)
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

---

## Data Dictionary 

### Features

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

- `amount_tsh` : Total static head (amount water available to waterpoint)
- `date_recorded` : The date the row was entered
- `funder` : Who funded the well
- `gps_height` : Altitude of the well
- `installer` : Organization that installed the well
- `longitude` : GPS coordinate
- `latitude` : GPS coordinate
- `wpt_name` : Name of the waterpoint if there is one
- `num_private` :  
- `basin` : Geographic water basin
- `subvillage` : Geographic location
- `region` : Geographic location
- `region_code` : Geographic location (coded)
- `district_code` : Geographic location (coded)
- `lga` : Geographic location
- `ward` : Geographic location
- `population` : Population around the well
- `public_meeting` : True/False
- `recorded_by` : Group entering this row of data
- `scheme_management` : Who operates the waterpoint
- `scheme_name` : Who operates the waterpoint
- `permit` : If the waterpoint is permitted
- `construction_year` : Year the waterpoint was constructed
- `extraction_type` : The kind of extraction the waterpoint uses
- `extraction_type_group` : The kind of extraction the waterpoint uses
- `extraction_type_class` : The kind of extraction the waterpoint uses
- `management` : How the waterpoint is managed
- `management_group` : How the waterpoint is managed
- `payment` : What the water costs
- `payment_type` : What the water costs
- `water_quality` : The quality of the water
- `quality_group` : The quality of the water
- `quantity` : The quantity of water
- `quantity_group` : The quantity of water
- `source` : The source of the water
- `source_type` : The source of the water
- `source_class` : The source of the water
- `waterpoint_type` : The kind of waterpoint
- `waterpoint_type_group` : The kind of waterpoint

### Labels

There are three possible values:

- `functional` : the waterpoint is operational and there are no repairs needed
- `functional needs repair` : the waterpoint is operational, but needs repairs
- `non functional` : the waterpoint is not operational

--- 

## Generate a submission

Your code to generate a submission file may look like this:

```python
# estimator is your model or pipeline, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = estimator.predict(X_test)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
```

If you're working locally, the csv file is saved in the same directory as your notebook.

If you're using Google Colab, you can use this code to download your submission csv file.

```python
from google.colab import files
files.download('your-submission-filename.csv')
```

---
---

In [1]:
# Musketeer imports
import pandas as pd
import numpy as np

In [2]:
# Logistic Regression Imports
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [61]:
# Set pandas display options to allow for more columns and rows
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)

---

#### Read the Tanzania Waterpumps data

- train_features.csv : the training set features
- train_labels.csv : the training set labels
- test_features.csv : the test set features
- sample_submission.csv : a sample submission file in the correct format

In [3]:
# Read the Tanzania Waterpumps data
train_features = pd.read_csv('../data/waterpumps/train_features.csv')
train_labels = pd.read_csv('../data/waterpumps/train_labels.csv')
test_features = pd.read_csv('../data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('../data/waterpumps/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

---

## 214 Assignment

In [68]:
train_features.head(24)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
9,46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [69]:
# Look at datatypes
train_features.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [67]:
# Look at non-numeric / categorical features
train_features.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,59400,1,GeoData Consultants Ltd,59400
public_meeting,56066,2,True,51011
permit,56344,2,True,38852
source_class,59400,3,groundwater,45794
management_group,59400,5,user-group,52490
quantity_group,59400,5,enough,33186
quantity,59400,5,enough,33186
waterpoint_type_group,59400,6,communal standpipe,34625
quality_group,59400,6,good,50818
payment_type,59400,7,never pay,25348


---

#### Some ideas based on initial data exploration

- Drop redundant features 
  - [ ] extraction_type / extraction_type_group
  - [ ] quantity / quantity_group
  - [ ] source / etc.
  - [ ] waterpoint_type
- Drop useless features
  - [ ] recorded_by
- Datatypes
  - [ ] Find / replace null values that were not read in as NaN
  - [ ] Convert date to datetime

---

#### Do train/validate/test split with the Tanzania Waterpumps data.

In [23]:
# Split up the train data into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_labels, random_state=92)

In [29]:
# Look at shape of each
data_list = {
    "X_train": X_train,
    "X_valid": X_valid,
    "y_train": y_train,
    "y_valid": y_valid,
}

def shape_printer():
    for d in data_list:
        print(f"{d}")
        print(f"{data_list[d].shape[0]} rows")
        print(f"{data_list[d].shape[1]} columns")
        print()

shape_printer()

X_train
36192 rows
39 columns

X_valid
14850 rows
40 columns

y_train
44550 rows
2 columns

y_valid
14850 rows
2 columns



In [ ]:
# Check for null values in X_train
X_train.isnull().sum()  # Looks like "scheme_name" should be dropped

In [26]:
def drop_scheme_name_and_na(data):
    """
    Returns dataframe without "scheme_name" column or rows with null values.
    """
    return data.drop("scheme_name", axis=1).dropna()

In [27]:
# Drop nulls from X_train and see resulting shape
X_train = drop_scheme_name_and_na(X_train)
print(X_train.shape)

(36192, 39)


In [30]:
# Drop nulls from X_valid and see resulting shape
X_valid = drop_scheme_name_and_na(X_valid)
print(X_valid.shape)

(12096, 39)


In [ ]:
# Confirm null values have been dropped
X_train.isnull().sum()

In [ ]:
# Confirm null values have been dropped
X_valid.isnull().sum()

## TODO: figure out how to drop the same records from target datasets

In [86]:
# Drop na from y_train
y_train.shape

(44550, 2)

---

#### Begin with baselines for classification.

In [41]:
# Get baseline accuracy score using the mode
y_train["status_group"].value_counts()  # The majority class : functional (good!)

functional                 24302
non functional             17046
functional needs repair     3202
Name: status_group, dtype: int64

In [51]:
# Create list of the majority class, length of target train
y_pred_base = pd.Series(["functional"] * len(y_train))
y_pred_base.head()

0    functional
1    functional
2    functional
3    functional
4    functional
dtype: object

In [55]:
y_train["status_group"].head()

12399        functional
15969    non functional
22944        functional
54034        functional
53053    non functional
Name: status_group, dtype: object

In [57]:
# Calculate accuracy score of baseline model for y_train
accuracy_score(y_train["status_group"], y_pred_base)

0.5454994388327722

> train accuracy score baseline of 0.5458

In [59]:
# Create list of the majority class, length of target validation
y_valid_pred_base = pd.Series(["functional"] * len(y_valid))
accuracy_score(y_valid["status_group"], y_valid_pred_base)  # accuracy of baseline model for y_valid

0.5358249158249159

> validation accuracy score baseline of 0.5345

---

#### Use scikit-learn for logistic regression.

In [74]:
# Choose a few features
features = [
    "permit",
    "source_class",
    "quality_group",
    "extraction_type_group",
]

In [76]:
# Create filtered training set based on features
X_train_feat = X_train[features]
X_valid_feat = X_valid[features]

In [77]:
# Instantiate encoder object
encoder = ce.OneHotEncoder(use_cat_names=True)

# Fit / transform X with encoder
X_train_feat_encoded = encoder.fit_transform(X_train_feat)
X_valid_feat_encoded = encoder.transform(X_valid_feat)

In [78]:
# Instantiate imputer object
imputer = SimpleImputer()

# Impute the data to fill in null values
X_train_feat_imputed = imputer.fit_transform(X_train_feat_encoded)
X_valid_feat_imputed = imputer.transform(X_valid_feat_encoded)

In [81]:
# Instantiate model object
log_reg_cv = LogisticRegressionCV(cv=5, n_jobs=-1, random_state=92)

In [83]:
y_train.head()

,id,status_group
12399,51067,functional
15969,29559,non functional
22944,59915,functional
54034,28479,functional
53053,21489,non functional


In [84]:
# Fit the model to the training data
log_reg_cv.fit(X_train_feat_imputed, y_train["status_group"])
print('Validation Accuracy', log_reg_cv.score(X_valid_feat_imputed, y_valid["status_group"]))

ValueError: Found input variables with inconsistent numbers of samples: [36192, 44550]

---

#### Get your validation accuracy score.

---

#### Submit your predictions to our Kaggle competition.

#### Commit your notebook to your fork of the GitHub repo.